### In Sample Evaluation And Cross Validation

The purpose of this notebook is evaluation my own cross validation with out the cross validation function of skealer library for the the spam filter model.But first lets load the dataset and the important libraries for our task.

In [4]:
import numpy as np
import pandas as pd
import sklearn

In [5]:
data_path = ("https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/"
             "master/sms_spam_collection/SMSSpamCollection"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['spam', 'message']

# Enumerate our spammy keywords.
keywords = ['click', 'offer', 'winner', 'buy', 'free', 'cash', 'urgent']

for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

sms_raw['allcaps'] = sms_raw.message.str.isupper()
sms_raw['spam'] = (sms_raw['spam'] == 'spam')

In [6]:
sms_raw.head()

,spam,message,click,offer,winner,buy,free,cash,urgent,allcaps
0,False,"Go until jurong point, crazy.. Available only ...",False,False,False,False,False,False,False,False
1,False,Ok lar... Joking wif u oni...,False,False,False,False,False,False,False,False
2,True,Free entry in 2 a wkly comp to win FA Cup fina...,False,False,False,False,False,False,False,False
3,False,U dun say so early hor... U c already then say...,False,False,False,False,False,False,False,False
4,False,"Nah I don't think he goes to usf, he lives aro...",False,False,False,False,False,False,False,False


In [7]:
no_folds = 14
pop_size = 5572
sample_size = int(pop_size/no_folds)
random_data = sms_raw.sample(n = 5572, axis = 0)
intervals = np.linspace(0, pop_size, no_folds + 1, dtype=int)

In [9]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
from sklearn.metrics import confusion_matrix
holdout = pd.DataFrame()
train_data = pd.DataFrame()
cross_validation = []
confus_matrix = []
for i in range(len(intervals) - 1):
    holdout = random_data[intervals[i]: intervals[i + 1]]
    if i == 0:
        train_data = random_data[intervals[i + 1]: intervals[-1]]
    elif (i > 0) & (i < (len(intervals) - 1)):
            train_data = pd.concat([random_data[intervals[0]:intervals[i]],random_data[intervals[i + 1]:intervals[-1]]])
    else:
        train_data = random_data[intervals[0]: intervals[i]]
    training_data = train_data[keywords + ['allcaps']]
    training_target = train_data['spam'] 
    test_data = holdout[keywords + ['allcaps']]
    test_target = holdout['spam']
    bnb.fit(training_data, training_target)
    predicted = bnb.predict(test_data)
    cross_validation.append(bnb.fit(training_data, training_target).score(test_data, test_target))
    confus_matrix.append(confusion_matrix(test_target, predicted))


In [10]:
cross_validation

[0.8793969849246231,
 0.8844221105527639,
 0.8919597989949749,
 0.8994974874371859,
 0.8668341708542714,
 0.914572864321608,
 0.914572864321608,
 0.9045226130653267,
 0.864321608040201,
 0.8919597989949749,
 0.8919597989949749,
 0.8844221105527639,
 0.8894472361809045,
 0.9045226130653267]